In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt

# Preparing Dataset

In [8]:
dataset_path = '/content/drive/MyDrive/datasets/7-7-faces'
width = height = 224

image_data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    validation_split=0.1
)

train_data = image_data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    subset='training',
    shuffle=True,
    batch_size=32
)

validation_data = image_data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    subset='validation',
    shuffle=False,
    batch_size=32
)


Found 1227 images belonging to 14 classes.
Found 131 images belonging to 14 classes.


# Creat Model

In [12]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(width, height, 3),
    pooling='avg'
)

9406464/9406464 [==============================] - 2s 0us/step


In [13]:
for layer in base_model.layers[0:-4]:
  layer.trainable = False

In [17]:
model = Sequential([
    base_model,
    layers.Dense(14, activation='softmax')
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 dense (Dense)               (None, 14)                17934     
                                                                 
Total params: 2275918 (8.68 MB)
Trainable params: 430094 (1.64 MB)
Non-trainable params: 1845824 (7.04 MB)
_________________________________________________________________


In [19]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy,
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_data,
          validation_data=validation_data,
          epochs=32)